In [ ]:
!pip install jcopdl==1.1.10
!pip install gdown

  Preparing metadata (setup.py) ... done
  Created wheel for jcopdl: filename=jcopdl-1.1.10-py2.py3-none-any.whl size=17915 sha256=041ffd23749e10399defa2a5017cf3f27cdfdb8cc2ecd6aa0627516efa12509d
  Stored in directory: /root/.cache/pip/wheels/0b/f2/20/aac0878bab38dc24137a63e0977201d72a5f892039d6b38885
Successfully built jcopdl


KeyboardInterrupt: 

In [ ]:
!gdown https://drive.google.com/uc?id=1WmEmUz6l3jXpV-znFu3EqqjTn4kf2OcJ

In [ ]:
import jcopdl
import numpy as np
import matplotlib.pyplot as plt
import io


In [ ]:
!unzip /content/sendi-full.zip

In [ ]:
!mv /content/sendi/data /content/
!rm -r /content/sendi

In [ ]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

#Dataset & Dataloader

In [ ]:
map5to3 = {0:0, 1:0, 2:0, 3:1, 4:2}

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

bs = 32
crop_size = 224

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet standards
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_set = datasets.ImageFolder("data/train", transform=train_transform, target_transform=lambda y: map5to3[y])
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=4)

val_set = datasets.ImageFolder("data/val", transform=test_transform, target_transform=lambda y: map5to3[y])
valloader = DataLoader(val_set, batch_size=bs, shuffle=False, num_workers=4)

test_set = datasets.ImageFolder("data/test", transform=test_transform, target_transform=lambda y: map5to3[y])
testloader = DataLoader(test_set, batch_size=bs, shuffle=False)


In [ ]:
label2cat = ["class0_1_2", "class3", "class4"]
label2cat

#Arsitektur & Config

In [ ]:
from torchvision.models import resnet18

resnet = resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
num_ftrs = resnet.fc.in_features

In [ ]:
resnet

In [ ]:
class CustomResnet18(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.resnet = resnet18(pretrained=True)
        self.frezee()
        self.resnet.fc = nn.Linear(num_ftrs, output_size)

    def frezee(self):
        for param in self.resnet.parameters():
            param.requires_grad = False

    def unfrezee(self):
        for param in self.resnet.parameters():
            param.requires_grad = True

    def forward(self, x):
        x = self.resnet(x)
        return x


In [ ]:
config = set_config({
    "output_size" : len(label2cat),
    "batch_size" : bs,
    "crop_size" : crop_size,
})

#Phase1 : Adaptation (lr standard + patience kecil)

In [ ]:
model = CustomResnet18(config.output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config,early_stop_patience=2, outdir="model")

In [ ]:
from collections import Counter

num_classes = 3
counts = Counter(train_set.targets)  # ImageFolder punya .targets
w = np.array([counts[c] for c in range(num_classes)], dtype=np.float64)
w = w.sum() / (w + 1e-6)
w = (w / w.mean()).astype(np.float32)

class_weights = torch.tensor(w).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)


In [ ]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)

        if mode == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [ ]:
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn("test", test_set, testloader, model, criterion, optimizer, device)

    # Logging
    callback.log(train_cost, test_cost, train_score, test_score)

    # Checkpoint
    callback.save_checkpoint()

    # Runtime Plotting
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()

    # Early Stopping
    if callback.early_stopping(model, monitor="test_score"):
        callback.plot_cost()
        callback.plot_score()
        break

#Phase 2 : Fine-tuning (lr kecil, patience ditambah)

In [ ]:
model.unfrezee()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
callback.reset_early_stop()
callback.early_stop_patience = 5


In [ ]:
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        val_cost,   val_score   = loop_fn("test",  val_set,   valloader,   model, criterion, optimizer, device)

    # log & plot (pakai val)
    callback.log(train_cost, val_cost, train_score, val_score)
    callback.save_checkpoint()
    callback.cost_runtime_plotting(); callback.score_runtime_plotting()

    if callback.early_stopping(model, monitor="test_score"):  # 'test_score' di callback = kolom ke-4, kita isi val_score
        callback.plot_cost(); callback.plot_score()
        break

# Predict

In [ ]:
# === EVALUASI (3 kelas) ===
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = outputs.argmax(1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

class_names = ['Healthy', 'Moderate', 'Severe']

print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))

cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.tight_layout()
plt.show()


In [ ]:
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)


with torch.no_grad():
  model.eval()
  output = model(feature)
  preds = torch.argmax(output, dim=1)

print("Predicted:", preds[:10].cpu().numpy())
print("Ground truth:", target[:10].cpu().numpy())

# Sanity Check



In [ ]:
def convert_to_label(x):
    return label2cat[int(x)]

def inverse_norm(img):
  img[0, :, :] = img[0, :, :] * 0.229 + 0.485
  img[1, :, :] = img[1, :, :] * 0.224 + 0.456
  img[2, :, :] = img[2, :, :] * 0.225 + 0.406
  return img

In [ ]:
import torch, numpy as np, matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Nama kelas 3-class
class_names = ['Healthy', 'Moderate', 'Severe']
idx2name = dict(enumerate(class_names))

def inverse_norm(img):
    img = img.clone()
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std  = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    img = img * std + mean
    return img.clamp(0, 1)

# ===== kumpulkan SEMUA gambar + prediksi dari testloader =====
model.eval()
imgs_all, labels_all, preds_all = [], [], []
with torch.no_grad():
    for x, y in testloader:
        p = model(x.to(device)).argmax(1).cpu()
        imgs_all.extend(x.cpu())
        labels_all.extend(y.cpu())
        preds_all.extend(p)

N = len(imgs_all)
print(f"Total test images: {N}")

# ===== buat PDF multi-halaman =====
per_page = 36
cols = 6
rows = (per_page + cols - 1)//cols

pdf_path = "test_predictions.pdf"
with PdfPages(pdf_path) as pdf:
    for start in range(0, N, per_page):
        end = min(start + per_page, N)
        fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
        axes = np.array(axes).reshape(rows, cols)

        for i in range(rows*cols):
            ax = axes[i // cols, i % cols]
            idx = start + i
            if idx < end:
                img = inverse_norm(imgs_all[idx]).permute(1,2,0).numpy()
                y = int(labels_all[idx]); p = int(preds_all[idx])
                ax.imshow(img)
                ax.set_title(f"Label: {idx2name[y]}\nPred: {idx2name[p]}",
                             color=('g' if y==p else 'r'), fontsize=10)
                ax.axis('off')
            else:
                ax.axis('off')

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Selesai. File disimpan: {pdf_path}")

# (opsional) tampilkan halaman pertama di notebook
from math import ceil
first_end = min(per_page, N)
fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
axes = np.array(axes).reshape(rows, cols)
for i in range(rows*cols):
    ax = axes[i // cols, i % cols]
    if i < first_end:
        img = inverse_norm(imgs_all[i]).permute(1,2,0).numpy()
        y = int(labels_all[i]); p = int(preds_all[i])
        ax.imshow(img)
        ax.set_title(f"Label: {idx2name[y]}\nPred: {idx2name[p]}",
                     color=('g' if y==p else 'r'), fontsize=10)
        ax.axis('off')
    else:
        ax.axis('off')
plt.tight_layout(); plt.show()


In [ ]:
try:
    # kalau wrapper-mu menyimpan backbone di atribut .resnet
    in_f = model.resnet.fc.in_features
    if getattr(model.resnet.fc, 'out_features', None) != 3:
        model.resnet.fc = nn.Linear(in_f, 3).to(device)
except AttributeError:
    # fallback kalau nama atribut berbeda
    print("Backbone bukan di model.resnet; lihat Cara robust di bawah.")

# 1) fungsi bantu hitung distribusi label dari dataset apapun (ImageFolder / wrapper)
from collections import Counter
def count_labels(ds):
    if hasattr(ds, "targets"):  # ImageFolder atau wrapper yg expose .targets
        ys = ds.targets
    else:
        ys = [ds[i][1] for i in range(len(ds))]
    return Counter(ys)

print("Train dist:", count_labels(train_set))
print("Val   dist:", count_labels(val_set))
print("Test  dist:", count_labels(test_set))


In [ ]:
# ===== 10 gambar per kelas (total 30), seimbang by TRUE LABEL =====
import math, numpy as np

class_names = ['Healthy', 'Moderate', 'Severe']
idx2name = dict(enumerate(class_names))

# 1) Kumpulkan indeks per kelas
idx_by_cls = {c: [i for i, y in enumerate(labels_all) if int(y) == c] for c in range(3)}
for c in idx_by_cls:
    np.random.shuffle(idx_by_cls[c])  # acak biar variatif

# 2) Ambil 10 per kelas (atau kurang kalau datanya tidak cukup)
k_per_class = min(10, *[len(idx_by_cls[c]) for c in range(3)])
balanced_idx = []
for i in range(k_per_class):
    for c in range(3):  # urutan: 0,1,2,0,1,2,...
        balanced_idx.append(idx_by_cls[c][i])

print({class_names[c]: len(idx_by_cls[c]) for c in range(3)})
print(f"Menampilkan {k_per_class} per kelas → total {len(balanced_idx)} gambar")

# 3) Plot satu halaman (30 gambar → 6x5)
cols = 6
rows = math.ceil(len(balanced_idx) / cols)
fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
axes = np.array(axes).reshape(rows, cols)

for i in range(rows * cols):
    ax = axes[i // cols, i % cols]
    if i < len(balanced_idx):
        idx = balanced_idx[i]
        img = inverse_norm(imgs_all[idx]).permute(1, 2, 0).numpy()
        y = int(labels_all[idx]); p = int(preds_all[idx])
        ax.imshow(img)
        ax.set_title(f"Label: {idx2name[y]}\nPred: {idx2name[p]}",
                     color=('g' if y == p else 'r'), fontsize=10)
        ax.axis('off')
    else:
        ax.axis('off')

fig.suptitle("Balanced test predictions (10 per class)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# ==== SAVE ARTIFACTS ====
import torch, json, os
from pathlib import Path

save_dir = Path("export")
save_dir.mkdir(parents=True, exist_ok=True)

# pastikan class_names & transform stats sama dgn yang dipakai saat test
class_names   = ['Healthy', 'Moderate', 'Severe']
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]
crop_size     = 224   # samakan dgn notebook
resize_size   = 256

# 1) Simpan state_dict + metadata (format PyTorch)
artifact = {
    "arch": "resnet18",
    "state_dict": (model.resnet.state_dict() if hasattr(model, "resnet") else model.state_dict()),
    "class_names": class_names,
    "mean": imagenet_mean,
    "std": imagenet_std,
    "preprocess": {"resize": resize_size, "center_crop": crop_size, "grayscale_to_rgb": True},
}
torch.save(artifact, save_dir / "resnet18_3class_best.pt")
print("Saved:", save_dir / "resnet18_3class_best.pt")

# 2) (opsional) Simpan TorchScript (portable)
model_cpu = (model.resnet if hasattr(model, "resnet") else model).cpu().eval()
example = torch.randn(1, 3, crop_size, crop_size)
traced = torch.jit.trace(model_cpu, example)
traced.save(str(save_dir / "resnet18_3class_traced.pt"))
print("Saved:", save_dir / "resnet18_3class_traced.pt")

# 3) Simpan class_names & preprocess juga ke JSON (berguna untuk app)
with open(save_dir / "class_names.json", "w") as f:
    json.dump(class_names, f)
with open(save_dir / "preprocess.json", "w") as f:
    json.dump({
        "resize": resize_size, "center_crop": crop_size,
        "mean": imagenet_mean, "std": imagenet_std,
        "grayscale_to_rgb": True
    }, f, indent=2)


In [ ]:
# === Inference utilities ===
import torch
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet18

def _build_transform_from_meta(meta: dict):
    ops = []
    if meta["preprocess"].get("grayscale_to_rgb", True):
        ops.append(transforms.Grayscale(num_output_channels=3))
    if meta["preprocess"].get("resize", None):
        ops.append(transforms.Resize(meta["preprocess"]["resize"]))
    if meta["preprocess"].get("center_crop", None):
        ops.append(transforms.CenterCrop(meta["preprocess"]["center_crop"]))
    ops += [
        transforms.ToTensor(),
        transforms.Normalize(mean=meta["mean"], std=meta["std"]),
    ]
    return transforms.Compose(ops)

def load_checkpoint_model(ckpt_path="export/resnet18_3class_best.pt", device="cpu"):
    """
    Load checkpoint (.pt / .pth) -> bangun resnet18(num_classes=3) -> load_state_dict
    """
    ckpt = torch.load(ckpt_path, map_location=device)   # <-- torch.load (BUKAN torch.jit.load)
    class_names = ckpt["class_names"]
    model = resnet18(weights=None, num_classes=len(class_names))
    model.load_state_dict(ckpt["state_dict"])
    model.eval().to(device)
    tfm = _build_transform_from_meta(ckpt)
    return model, class_names, tfm

@torch.no_grad()
def predict_path(img_path: str, model, class_names, tfm, device="cpu"):
    img = Image.open(img_path).convert("RGB")
    x = tfm(img).unsqueeze(0).to(device)
    logits = model(x)
    probs = torch.softmax(logits, dim=1).squeeze(0).cpu().tolist()
    pred_idx = int(torch.argmax(logits, dim=1).item())
    return {
        "pred_idx": pred_idx,
        "pred_label": class_names[pred_idx],
        "probs": {class_names[i]: float(p) for i, p in enumerate(probs)}
    }

@torch.no_grad()
def predict_bytes(file_bytes: bytes, model, class_names, tfm, device="cpu"):
    """
    Berguna untuk FastAPI nanti: terima bytes upload -> prediksi
    """
    img = Image.open(io.BytesIO(file_bytes)).convert("RGB")
    x = tfm(img).unsqueeze(0).to(device)
    logits = model(x)
    probs = torch.softmax(logits, dim=1).squeeze(0).cpu().tolist()
    pred_idx = int(torch.argmax(logits, dim=1).item())
    return {
        "pred_idx": pred_idx,
        "pred_label": class_names[pred_idx],
        "probs": {class_names[i]: float(p) for i, p in enumerate(probs)}
    }


In [ ]:
# === Smoke test: load checkpoint & prediksi 1 gambar ===
import os, io

device = "cpu"  # untuk test cepat
model_inf, class_names_inf, tfm_inf = load_checkpoint_model(
    "export/resnet18_3class_best.pt", device=device
)
print("Loaded. Classes:", class_names_inf)


try:
    sample_path, _ = test_set.samples[0]
except NameError:
    sample_path = "data/test/Healthy/some_image.png"

assert os.path.exists(sample_path), f"not found: {sample_path}"

res = predict_path(sample_path, model_inf, class_names_inf, tfm_inf, device=device)
print("Sample prediction:", res)


In [ ]:
# === Full test evaluation from saved artifact ===
import torch, json
import numpy as np
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns, matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

# load ulang model & transform dari checkpoint
model_eval, class_names_eval, tfm_eval = load_checkpoint_model(
    "export/resnet18_3class_best.pt", device=device
)

model_eval.eval()
all_labels, all_preds = [], []

with torch.no_grad():
    for xb, yb in testloader:
        xb = xb.to(device)
        logits = model_eval(xb)
        preds = logits.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(yb.numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"Test accuracy: {acc:.4f}")
print(classification_report(all_labels, all_preds, target_names=class_names_eval))

# simpan confusion matrix ke gambar
cm = confusion_matrix(all_labels, all_preds, labels=[0,1,2])
Path("assets").mkdir(exist_ok=True)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_eval, yticklabels=class_names_eval)
plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.tight_layout()
plt.savefig("assets/cm_test.png", dpi=200)
plt.show()

# simpan report ke JSON
Path("export").mkdir(exist_ok=True)
with open("export/test_report.json","w") as f:
    json.dump({
        "accuracy": float(acc),
        "report": classification_report(all_labels, all_preds, target_names=class_names_eval, output_dict=True)
    }, f, indent=2)

print("Saved -> assets/cm_test.png  &  export/test_report.json")


#akhir